## Q1

Type 1 error ~ FP ( 실제는 False인데, True라 판단)

Type 2 error ~ FN ( 실제는 True인데, False라 판단)

Type 1 error와 Type 2 error가 서로 trade off 관계에 있듯이 FP와 FN도 서로 trade off 관계에 있다.

따라서, threshold = 0.3으로 낮추면, FN이 작아지고, FP가 커진다.
즉, Type 1 error 가 더 커진다.

## Q2-1

Accuracy(정확도: 전체 예측 중 옳게 말한 결과의 비율) = (TP+TN) / (TP+TN+FP+FN)

Precision(정밀도: True라고 예측한 것 중 실제로 True의 비율) = TP / (TP+FP)

Recall(재현율: 실제 True 중 True라고 예측한 것의 비율) = TP / (TP+FN)

## Q2-2

암을 진단하는 문제: 관심사건 = 암이다.

threshold를 늘린다는 것은 웬만하면 암으로 진단하지 않겠다는 것을 의미한다.

이는 비합리적이다. 

왜냐하면, 암에 걸린 사람을 암이 아니라고 진단하는 것이 암에 걸리지 않은 사람을 암이라고 진단하는 것보다 더 심각한 오류이기 때문이다.

(물론, 암이라고 오진단을 받고 막대한 병원비 때문에 목숨을 걸 정도로, 생명보다 돈이 중요한 사람에게는 threshold를 낮추는 게
더 비합리적일 것이다.)

## Q3

In [18]:
# import
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable

In [59]:
# For reproducibility 
torch.manual_seed(1)

In [3]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data', 
							train = True, 
							transform = transforms.ToTensor(), 
							download = True) 

test_dataset = dsets.MNIST(root ='./data', 
						train = False, 
						transform = transforms.ToTensor()) 

# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
										batch_size = batch_size, 
										shuffle = True) 

test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
										batch_size = batch_size, 
										shuffle = False) 


In [5]:
class LogisticRegression(nn.Module): 
	def __init__(self, input_size, num_classes): 
		super(LogisticRegression, self).__init__() 
		self.linear = nn.Linear(input_size, num_classes) 

	def forward(self, x): 
		out = self.linear(x) 
		return out 


In [6]:
model = LogisticRegression(input_size, num_classes) 


In [7]:
criterion = nn.CrossEntropyLoss() #cost function
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) #optimization 방식: SGD

In [9]:
# Training the Model 
for epoch in range(num_epochs): 
	for i, (images, labels) in enumerate(train_loader): 
		images = Variable(images.view(-1, 28 * 28)) 
		labels = Variable(labels) 

		# Forward + Backward + Optimize  
		optimizer.zero_grad()  #초기화
		outputs = model(images)  #결과
		loss = criterion(outputs, labels) 
		loss.backward() 
		optimizer.step() #update

		if (i + 1) % 100 == 0: 
			print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
				% (epoch + 1, num_epochs, i + 1, 
					len(train_dataset) // batch_size, loss.data)) 


Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.0943
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.0331
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.9534
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.8604
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8261
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7157
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7001
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.5802
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5340
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5098
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4562
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.3380
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3218
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.3357
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.2131
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2959
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2757
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2354
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.1801
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1512
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [10]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
	images = Variable(images.view(-1, 28 * 28)) 
	outputs = model(images) 
	_, predicted = torch.max(outputs.data, 1) 
	total += labels.size(0) 
	correct += (predicted == labels).sum() t

print('Accuracy of the model on the 10000 test images: % d %%' % ( 
			100 * correct / total)) 


Accuracy of the model on the 10000 test images:  83 %


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


## Q4-1

### optim.sgd

In [60]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 
class LinearRegressionModel(nn.Module):    
    def __init__(self):        
        super().__init__()        
        self.linear = nn.Linear(1, 1) 
        
    def forward(self, x):        
        return self.linear(x)    

In [66]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.066, b: -0.150 Cost: 0.003239
Epoch  200/1000 W: 1.052, b: -0.118 Cost: 0.002002
Epoch  300/1000 W: 1.041, b: -0.093 Cost: 0.001237
Epoch  400/1000 W: 1.032, b: -0.073 Cost: 0.000764
Epoch  500/1000 W: 1.025, b: -0.057 Cost: 0.000472
Epoch  600/1000 W: 1.020, b: -0.045 Cost: 0.000292
Epoch  700/1000 W: 1.016, b: -0.035 Cost: 0.000180
Epoch  800/1000 W: 1.012, b: -0.028 Cost: 0.000111
Epoch  900/1000 W: 1.010, b: -0.022 Cost: 0.000069
Epoch 1000/1000 W: 1.008, b: -0.017 Cost: 0.000043


## another way

In [62]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]])  

In [68]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
lr=0.01

nb_epochs=1000
for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = (x_train * W) + b       

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        
    gradient=torch.sum(((W * x_train + b)- y_train) * x_train)
    b_update=torch.sum((W*x_train + b) - y_train)
    
    # 100번마다 로그 출력                 
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()        
        ))
    #cost gradient로 H(X) 개선
    W-=lr*gradient
    b-=lr*b_update

Epoch    0/1000 W: 0.000, b: 0.000 Cost: 4.666667
Epoch  100/1000 W: 0.887, b: 0.257 Cost: 0.009462
Epoch  200/1000 W: 0.921, b: 0.179 Cost: 0.004594
Epoch  300/1000 W: 0.945, b: 0.125 Cost: 0.002231
Epoch  400/1000 W: 0.962, b: 0.087 Cost: 0.001083
Epoch  500/1000 W: 0.973, b: 0.061 Cost: 0.000526
Epoch  600/1000 W: 0.981, b: 0.042 Cost: 0.000255
Epoch  700/1000 W: 0.987, b: 0.029 Cost: 0.000124
Epoch  800/1000 W: 0.991, b: 0.020 Cost: 0.000060
Epoch  900/1000 W: 0.994, b: 0.014 Cost: 0.000029
Epoch 1000/1000 W: 0.996, b: 0.010 Cost: 0.000014


## Q4-2

In [ ]:
#1 sgd

In [64]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 
class LinearRegressionModel(nn.Module):    
    def __init__(self):        
        super().__init__()        
        self.linear = nn.Linear(1, 1) 
        
    def forward(self, x):        
        return self.linear(x)    

In [67]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.066, b: -0.150 Cost: 0.003239
Epoch  200/1000 W: 1.052, b: -0.118 Cost: 0.002002
Epoch  300/1000 W: 1.041, b: -0.093 Cost: 0.001237
Epoch  400/1000 W: 1.032, b: -0.073 Cost: 0.000764
Epoch  500/1000 W: 1.025, b: -0.057 Cost: 0.000472
Epoch  600/1000 W: 1.020, b: -0.045 Cost: 0.000292
Epoch  700/1000 W: 1.016, b: -0.035 Cost: 0.000180
Epoch  800/1000 W: 1.012, b: -0.028 Cost: 0.000111
Epoch  900/1000 W: 1.010, b: -0.022 Cost: 0.000069
Epoch 1000/1000 W: 1.008, b: -0.017 Cost: 0.000043


In [69]:
#2 sgd with momentum

In [70]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 
class LinearRegressionModel(nn.Module):    
    def __init__(self):        
        super().__init__()        
        self.linear = nn.Linear(1, 1) 
        
    def forward(self, x):        
        return self.linear(x)    

In [73]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.007, b: -0.008 Cost: 0.000053
Epoch  200/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: -0.000 Cost: 0.000000


In [74]:
#3 adam

In [76]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 
class LinearRegressionModel(nn.Module):    
    def __init__(self):        
        super().__init__()        
        self.linear = nn.Linear(1, 1) 
        
    def forward(self, x):        
        return self.linear(x)    

In [77]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.525, b: -0.431 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.032 Cost: 0.000333
Epoch  200/1000 W: 0.990, b: 0.022 Cost: 0.000072
Epoch  300/1000 W: 0.993, b: 0.017 Cost: 0.000040
Epoch  400/1000 W: 0.995, b: 0.012 Cost: 0.000020
Epoch  500/1000 W: 0.997, b: 0.008 Cost: 0.000008
Epoch  600/1000 W: 0.998, b: 0.005 Cost: 0.000003
Epoch  700/1000 W: 0.999, b: 0.003 Cost: 0.000001
Epoch  800/1000 W: 0.999, b: 0.002 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


In [78]:
#4 rmsprop

In [79]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]]) 
class LinearRegressionModel(nn.Module):    
    def __init__(self):        
        super().__init__()        
        self.linear = nn.Linear(1, 1) 
        
    def forward(self, x):        
        return self.linear(x)    

In [80]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.615, b: -0.341 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.019 Cost: 0.000052
Epoch  200/1000 W: 0.996, b: 0.008 Cost: 0.000010
Epoch  300/1000 W: 0.999, b: 0.002 Cost: 0.000001
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 0.984, b: -0.016 Cost: 0.002620
Epoch  800/1000 W: 0.999, b: -0.001 Cost: 0.000006
Epoch  900/1000 W: 0.998, b: -0.002 Cost: 0.000030
Epoch 1000/1000 W: 0.996, b: -0.004 Cost: 0.000109
